<a href="https://colab.research.google.com/github/haiyinyang929/Statistical-Language-Processing-A4/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 4

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


## Honor Code
Course:        Statistical Language Processing - Summer 2024  
Assignment:   A4
Author(s):    Yip, Hiu Yan / Yang, Hai-Yin  
Honor Code:   We pledge that this program represents our own work,
and that we have not given or received unauthorized help
with this assignment.

## Part 1 Notebook Setup

- Install and import the packages.  
- Mount Google Drive and create two directories, one to save training
 checkpoints and one to save models.
- Set variable device, which need later:  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
!pip install google-colab
!pip install datasets transformers evaluate
!pip install torch
!pip install accelerate -U
!pip install transformers[torch]

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pya

In [ ]:
from google.colab import drive
from datasets import load_dataset
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
# mount Google Drive and create two directories, one to save training checkpoints and one to save models
drive.mount('/content/gdrive', force_remount=True)

# a new directory for training checkpoints
training_checkpoint_dir = "slp2-2024-assignment4-checkpoint"
!mkdir /content/gdrive/My\ Drive/$training_checkpoint_dir

# a new directory for saving models
models_dir = "slp2-2024-assignment4-models"
!mkdir /content/gdrive/My\ Drive/$models_dir

# navigate to your drive directory
%cd /content/gdrive/My\ Drive/

Mounted at /content/gdrive
mkdir: cannot create directory ‘/content/gdrive/My Drive/slp2-2024-assignment4-checkpoint’: File exists
mkdir: cannot create directory ‘/content/gdrive/My Drive/slp2-2024-assignment4-models’: File exists
/content/gdrive/My Drive


In [ ]:
# set up variable device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Part 2 Prepare the Training Data - Tokenization

Prepare the Training Data: <br>
(a) Load the emotion subset of the tweet dataset <br>
(b) Tokenize the entire dataset <br>
(c) Create small data splits for cpu training <br>

For dataset loading instructions, see the "Use this dataset" button in the right column of the HuggingFace page.

Until get all bugs out of your code, you should use a cpu:
Runtime -> Change runtime type -> CPU

DistilBERT: https://huggingface.co/distilbert/distilbert-base-uncased

Hugging face dataset(tweet_eval): https://huggingface.co/datasets/cardiffnlp/tweet_eval

In [ ]:
# Load model directly
from transformers import AutoTokenizer

checkpoint = "distilbert/distilbert-base-uncased" # checkpoint from hugging face DistilBERT (uncased for case insensitive)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

dataset = load_dataset("cardiffnlp/tweet_eval", "emotion") # from hugging face datasets (emotion)

train_data = dataset["train"]
dev_data = dataset["validation"]
test_data = dataset["test"]

# processing the data
def tokenize_function(data):
    return tokenizer(data['text'], truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)



# details
"""
print(dataset)
for key, value in dataset.items():
    print(f"{key}: {value[0]}\n")
print(tokenized_dataset)
for key, value in tokenized_dataset.items():
    print(f"{key}: {value[0]}\n")
"""



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/1421 [00:00<?, ? examples/s]

'\nprint(dataset)\nfor key, value in dataset.items():\n    print(f"{key}: {value[0]}\n")\nprint(tokenized_dataset)\nfor key, value in tokenized_dataset.items():\n    print(f"{key}: {value[0]}\n")\n'

### Create a small datasets
Also, don’t need to use much training data just for debugging the code.
<br>
If device.type == "cpu", use only the first (50/10/10) of the tokenized train, dev, test splits.

In [ ]:
drive.mount('/content/gdrive')
!mkdir /content/gdrive/My\ Drive/training-sm

tokenized_train_sm = tokenized_dataset["train"].select(range(50))
tokenized_train_sm.save_to_disk("/content/gdrive/My Drive/training-sm")
print("save1")

tokenized_dev_sm = tokenized_dataset["validation"].select(range(10))
tokenized_dev_sm.save_to_disk("/content/gdrive/My Drive/training-sm")
print("save2")

tokenized_test_sm = tokenized_dataset["test"].select(range(10))
tokenized_test_sm.save_to_disk("/content/gdrive/My Drive/training-sm")
print("save3")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
mkdir: cannot create directory ‘/content/gdrive/My Drive/training-sm’: File exists


Saving the dataset (0/1 shards):   0%|          | 0/50 [00:00<?, ? examples/s]

save1


Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

save2


Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

save3


使用 Hugging Face 提供的 DataCollatorWithPadding 和已加載的 DistilBERT 分詞器（tokenizer），創建一個可以自動對輸入數據進行填充的工具

In [ ]:
from transformers import DataCollatorWithPadding
# pad all the examples to the length of the longest element when we batch elements together
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# for testing only, not use elsewhere (make sure data_collator is working):
samples = tokenized_train_sm[:50]

# extract "input_ids"
samples = {key: value for key, value in samples.items() if key in ["input_ids"]}
print(f"row: len of input_ids, col: len of the tokens in each ids: \n{[len(x) for x in samples['input_ids']]}\n")

# batch (padding)
batch = data_collator(samples)
{key: value.shape for key, value in batch.items()} # [50, 38]: 50 代表樣本數量，38 代表所有樣本的統一長度（經過填充後）

row: len of input_ids, col: len of the tokens in each ids: 
[26, 30, 25, 29, 17, 30, 23, 22, 22, 12, 29, 32, 33, 13, 20, 32, 36, 25, 34, 38, 22, 35, 25, 29, 12, 36, 7, 27, 25, 28, 22, 18, 24, 31, 20, 6, 35, 19, 17, 34, 37, 16, 33, 38, 37, 20, 17, 27, 35, 25]



{'input_ids': torch.Size([50, 38]), 'attention_mask': torch.Size([50, 38])}

## Part 3 Training configurations

### Load the pretrained distilbert-base-uncased model
- From training feature, get the labels list and create corresponding label-to-id dicts.  
- Instantiate a model from AutoModelForSequenceClassification

Setting up a pipeline for text classification using a pre-trained transformer model with the Hugging Face transformers library.

- 標籤名稱(eg: positive/negative)和數字 ID(0 ,1)進行相互映射
- 將一個通用的預訓練 Transformer 模型(DistilBERT)轉換為文本分類模型(AutoModelForSequenceClassification)，設定為識別並處理數據集中的分類模型。



In [ ]:
from transformers import Trainer, TrainingArguments

labels = tokenized_dataset["train"].features["label"].names
#print(labels)

num_labels = len(labels)
#print(num_labels)

label2id = {label: labels.index(label) for label in tokenized_dataset["train"].features["label"].names}

id2label = {id: label for label, id in label2id.items()}
print(id2label)

# load a pre-trained transformer model for sequence classification tasks(text classification), using the Hugging Face transformers library.
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,
                                                           num_labels=num_labels,
                                                           id2label=id2label,
                                                           label2id=label2id)

{0: 'anger', 1: 'joy', 2: 'optimism', 3: 'sadness'}


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Evaluation Setup

- define a `compute_metrics` function for the trainer object
- use logits and real labels to calculate the f1 score (using hugging face evaluater)


In [ ]:
import evaluate
import numpy as np
from torch.utils.data import DataLoader

# Trainer object can precit the output, output is the logits and labels
# e.g. eval_preds = trainer.predict(tokenized_train_sm)

def compute_metrics(eval_preds):
    metric = evaluate.load("f1")
    logits = eval_preds.predictions # predict
    labels = eval_preds.label_ids   # gold label

    # get the argmax item along each output vector
    predictions = np.argmax(logits, axis=-1) # maximum value with -1 axis (the last dimension)
    print(predictions)
    return metric.compute(predictions=predictions, references=labels, average='macro') # macro for equal weight to each class (overall score)

### Construct Trainer Object
- set up Training Arguments
- instantiate a trainer object with the required parameters
- can change the full dataset to  `train_dataset=tokenized_train_sm`,
    `eval_dataset=tokenized_dev_sm`  (small dataset for testing)

In [ ]:
import accelerate
# change the output_dir to models_dir
training_args = TrainingArguments(output_dir= models_dir,
                                  save_total_limit=2,
                                  load_best_model_at_end=True,
                                  eval_strategy="steps",
                                  save_strategy="steps",
                                  num_train_epochs=3,
                                  logging_strategy="steps",
                                  evaluation_strategy="steps")
# load_best_model_at_end=True, automatically load the best model at the end of training.

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### Train the model

**Trainer** 是 Hugging Face transformers 庫中，專門用於訓練 Transformer 類模型。

需要的參數arguments:
- model
- TrainingArgument（

- train_dataset: 訓練數據集

- eval_dataset: 驗證數據集

- data_collator（可選）: 用於批次處理的數據整理工具，負責將數據按批次組合並進行處理（填充）
- tokenizer（可選）

- compute_metrics（可選）:自定義函數，用來計算每個步驟的評估指標。（先前score f1）

- callbacks（可選）


In [ ]:
# full datasets
tokenized_dataset_train = tokenized_dataset["train"]
tokenized_dataset_dev = tokenized_dataset["validation"]

# train the small dataset for testing
"""
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_sm,
    eval_dataset=tokenized_dev_sm,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
"""

# train the full dataset
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_dev,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


### Launch training run

Important: move the model to the device before training:
`model.to(device)`

when you are ready to train with the full dataset, request a GPU:
**Runtime -> Change runtime type -> T4 GPU**

Training for the default 3 epochs will take roughly 2 minutes on a GPU.



In [ ]:
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

## Evaluation Setup

- return **eval_results**

In [ ]:
eval_results = trainer.evaluate()

print(eval_results)

## Save the Best Model

Save the model to the directory that was created for that purpose when mounting the drive.

After the model is saved, it is safe to delete the contents of the directory used for training. (this is to avoid running out of space.)

Check often how much of your google drive quota you are using.

Check the contents of the trash and empty often.

In [ ]:
# save the model
trainer.save_model(models_dir)

In [ ]:
# delete the contents
!rm -r /content/gdrive/MyDrive/slp2-2024-assignment4-checkpoint

## Load saved model
- should now be able to load your model in the same way you loaded the "distilbert-base-uncased" model earlier  
- use AutoModelForSequenceClassification. But this
time you only need to provide the model location,  
- create a TextClassificationPipeline.

In [ ]:
# load the model
model = AutoModelForSequenceClassification.from_pretrained(models_dir)

In [ ]:
# text classification pipelie
from transformers import pipeline

classifier = pipeline(task="text-classification", model=model, tokenizer=tokenizer)

## Use the saved model to make our own inference

Use the pipeline created in the previous step to classify some tweets (at least 15). You can use tweets from the test set for this part.

For each tweet, print the model's prediction, the model's confidence score, and the tweet.

In [ ]:
# select 15 tweets from the test set

tweets = test_data.select(range(15))["text"]

for tweet in tweets:
  result = classifier(tweet)

  #print(result)

  print(f"Prediction: {result[0]['label']}")
  print(f"Confidence Score: {result[0]['score']}\n")
  print(f"Tweet: {tweet}")